In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import argparse
from tokenizers import SentencePieceBPETokenizer
from transformers import PreTrainedTokenizerFast
import argparse
import datetime
import pandas as pd
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/My Drive/Colab Notebooks/nlp_assignment2"

In [ ]:
def train_tokenizer(data_list, vocab_size=128000, model_name="Tokenizer3"):

    ## Change bos & eos
    bos_tok = "<sos>"
    eos_tok = "<end_of_sen>"

    ## Add basic characters to this below list, including numbers & special language characters.
    special_char = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]


    tokenizer = SentencePieceBPETokenizer()

    tokenizer.train_from_iterator(
        data_list,
        vocab_size = 128000,
        min_frequency = 5,
        special_tokens = ["<pad>", "<unk>", bos_tok, eos_tok, "<user>", "<assistant>"] + special_char,
        show_progress = True,
    )

    ## Don't forget to add special tokens.
    transformer_tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        bos_token = bos_tok,
        eos_token = eos_tok,
        unk_token = "<unk>",
        pad_token = "<pad>",
        mask_token = "<mask>",
        padding_side = "left",
        truncation_side = "right",
        additional_special_tokens = ["<user>", "<assistant>"],
        clean_up_tokenization_spaces = False,
    )

    transformer_tokenizer.save_pretrained(os.path.join(path, model_name))

In [ ]:
df = pd.read_csv(os.path.join(path, "sample_2.csv"))


In [ ]:
df.head()

,Unnamed: 0.1,source,filename,contents
0,150029,marathi.indiatimes.com,6783ef748f4aa70a042bcb361c916d79a13ff169ce0575...,हिन्दी महाराष्ट्र महाराष्ट्र विधानसभा निव...
1,1275975,rashtramat_articles,6612.txt,का झाली आपचे नेते अमित पालेकर यांना अटक? : ब...
2,768089,esakal.com,7ae5301012d937c838b23d08b35e34db5dfa8134f87ba4...,...
3,810594,esakal.com,86ac9f9c64063d66a4ae7f7c63524548ca562bab2b939c...,...
4,1434929,zeenews.india.com,ec92e8209c19ceeba0707436b737ff872196630f37da63...,...


In [ ]:
len(df["contents"].to_list())

289934

In [ ]:
### Executing Training Function to Train tokenizer
train_tokenizer(df["contents"].astype(str).to_list())

In [ ]:
### Testing Training Tokenizer
from transformers import AutoTokenizer

In [ ]:
## Load Tokenizers

tokenizer1 = AutoTokenizer.from_pretrained(os.path.join(path, "Tokenizer1"))
tokenizer2 = AutoTokenizer.from_pretrained(os.path.join(path, "Tokenizer2"))
tokenizer3 = AutoTokenizer.from_pretrained(os.path.join(path, "Tokenizer3"))
tokenizer4 = AutoTokenizer.from_pretrained(os.path.join(path, "Tokenizer4"))
tokenizer5 = AutoTokenizer.from_pretrained(os.path.join(path, "Tokenizer5"))


In [ ]:
## test Data



df = pd.read_csv(os.path.join(path, "test_sample.csv"))

In [ ]:
test_list = df["contents"].to_list()
test_list = test_list[:1000]

In [ ]:
print(len(tokenizer1.get_vocab()))
print(len(tokenizer2.get_vocab()))
print(len(tokenizer3.get_vocab()))
print(len(tokenizer4.get_vocab()))
print(len(tokenizer5.get_vocab()))

128001
128001
128001
128001
128001


In [ ]:
input_ids1= []
input_ids2= []
input_ids3= []
input_ids4= []
input_ids5= []

for text in tqdm(test_list):
  text = str(text)
  input_ids1.append(tokenizer1.encode(text))
  input_ids2.append(tokenizer2.encode(text))
  input_ids3.append(tokenizer3.encode(text))
  input_ids4.append(tokenizer4.encode(text))
  input_ids5.append(tokenizer5.encode(text))

# print(input_ids1)
# print(input_ids2)
# print(input_ids3)
# print(input_ids4)
# print(input_ids5)


100%|██████████| 1000/1000 [00:21<00:00, 46.47it/s]


In [ ]:
no_of_words = 0
for i in test_list:
   i = str(i)
   no_of_words += len(i.split(" "))

In [ ]:
no_of_words

813889

In [ ]:
# input_text = "तुम्ही लोकांना बऱ्याच वेळा नाही म्हणू इच्छित असाल.".split(" ")
# word_count  = len(input_text)

In [ ]:
sum(len(i) for i in input_ids1)

841817

In [ ]:

## Fertility Score calculations for all five tokenizers

fertility1 = sum(len(i) for i in input_ids1) / no_of_words
fertility2 = sum(len(i) for i in input_ids2) / no_of_words
fertility3 = sum(len(i) for i in input_ids3) / no_of_words
fertility4 = sum(len(i) for i in input_ids4) / no_of_words
fertility5 = sum(len(i) for i in input_ids5) / no_of_words



print(f"Fertility Score of tokenizer1: {fertility1}")
print(f"Fertility Score of tokenizer2: {fertility2}")
print(f"Fertility Score of tokenizer3: {fertility3}")
print(f"Fertility Score of tokenizer4: {fertility4}")
print(f"Fertility Score of tokenizer5: {fertility5}")


Fertility Score of tokenizer1: 1.0343142615270633
Fertility Score of tokenizer2: 1.0342307120504148
Fertility Score of tokenizer3: 1.0341533059176373
Fertility Score of tokenizer4: 1.0341987666622845
Fertility Score of tokenizer5: 1.0341545345864116


In [ ]:
input = "Hi, how are you?"
input

In [ ]:
pre_pro = "<bos><user>Hi, how are you?<asisstant>"
pre_pro

In [ ]:
final_reply = "<bos>Hi, how are you? I am great!<eos>"